# 05: EDA - Explore Night lights and Population

As I was working on the data preparation in the last notebook, I realized it will probably be pretty hard to tell the population change from satellite imagery since buildings usually stay for a while even if people are not living there anymore.

I will explore in this notebook if there is a correlation between change in night lights and the population to see if I can use this data to explore population decline.

Use census data and VIIRS Nighttime data from 2015 to 2020.



In [37]:
import ee
import time

In [3]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


Successfully saved authorization token.


### Get 2015 and 2020 images, export

In [41]:
i_date = '2015-01-01'
f_date = '2015-12-31'

# Get median image for year
night_light_2015 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(i_date, f_date).select('avg_rad').median()

# census data
census_2015 = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Count").filterDate(i_date, f_date).first()


i_date2 = '2020-01-01'
f_date2 = '2020-12-31'

# Get median image for year
night_light_2020 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(i_date2, f_date2).select('avg_rad').median()

# census data
census_2020 = ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Count").filterDate(i_date2, f_date2).first()

In [36]:
# Define regions

# Kanazawa to Yokohama
region1 = ee.Geometry.BBox(135.9259, 35.4698, 140.8698 , 36.6422)

# Kyushu
region2 = ee.Geometry.BBox(129.542275, 31.629761 ,131.981586 , 33.760804)

# Hiroshima/Osaka
region3 = ee.Geometry.BBox(130.987217, 33.760804, 136.611616, 35.448746)

# Shikoku
region4 = ee.Geometry.BBox(131.981586, 32.811406, 136.611616, 33.760804)

# Nagoya to Yokohama
region5 = ee.Geometry.BBox(136.6950, 34.4071, 140.6061, 35.4519)

# North of Tokyo to Niigata
region6 = ee.Geometry.BBox(136.5728, 36.6422, 141.0772, 38.3578)

# Sendai/Fukushima
region7 = ee.Geometry.BBox(139.5172, 38.3578, 142.0440, 41.5075)

# Hokkaido
region8 = ee.Geometry.BBox(139.4293, 41.5075, 145.6695, 45.5669)

all_regions = [region1, region2, region3, region4, region5, region6, region7, region8]


In [50]:
images_nightlights = [night_light_2015, night_light_2020]
years = [2015, 2020]

for i, region in enumerate(all_regions):
    for j, image in enumerate(images_nightlights):
        print(f'Processing: Region {i}, Year {years[j]}')
        task = ee.batch.Export.image.toDrive(image = image,
                                             fileNamePrefix = f'nightlights_{years[j]}_region{i}',
                                             region = region,
                                             scale = 1000, # adjust to Census data
                                             folder = 'NightLightImages')

        task.start()

        status = task.status()['state']

        while status != 'COMPLETED':
            print(f'Status: {status}')

            if status == 'FAILED':
                print(task.status())
                break

            time.sleep(10)
            status = task.status()['state']

images_census = [census_2015, census_2020]
years = [2015, 2020]

for i, region in enumerate(all_regions):
    for j, image in enumerate(images_census):
        print(f'Processing: Region {i}, Year {years[j]}')
        task = ee.batch.Export.image.toDrive(image = image,
                                             fileNamePrefix = f'census_{years[j]}_region{i}',
                                             region = region,
                                             scale = 1000,
                                             folder = 'NightLightImages')

        task.start()
        status = task.status()['state']
        while status != 'COMPLETED':
            print(f'Status: {status}')

            if status == 'FAILED':
                print(task.status())
                break

            time.sleep(10)
            status = task.status()['state']

Processing: Region 0, Year 2015
Status: READY
Status: RUNNING
Processing: Region 0, Year 2020
Status: READY
Status: RUNNING
Processing: Region 1, Year 2015
Status: READY
Status: RUNNING
Processing: Region 1, Year 2020
Status: READY
Status: RUNNING
Processing: Region 2, Year 2015
Status: READY
Status: RUNNING
Processing: Region 2, Year 2020
Status: READY
Status: RUNNING
Status: RUNNING
Processing: Region 3, Year 2015
Status: READY
Status: RUNNING
Processing: Region 3, Year 2020
Status: READY
Status: RUNNING
Processing: Region 4, Year 2015
Status: READY
Status: RUNNING
Processing: Region 4, Year 2020
Status: READY
Status: RUNNING
Processing: Region 5, Year 2015
Status: READY
Status: READY
Status: READY
Status: READY
Status: RUNNING
Status: RUNNING
Processing: Region 5, Year 2020
Status: READY
Status: READY
Status: RUNNING
Processing: Region 6, Year 2015
Status: READY
Status: RUNNING
Processing: Region 6, Year 2020
Status: READY
Status: RUNNING
Processing: Region 7, Year 2015
Status: READ

In [45]:
task = ee.batch.Export.image.toDrive(image = night_light_2015,
                                             fileNamePrefix = f'nightlights_{years[0]}_region{0}',
                                             region = all_regions[0],
                                             scale = 1000, # adjust to Census data
                                             folder = 'NightLightImages')

In [46]:
task.start()

In [49]:
task.status()

{'state': 'COMPLETED',
 'description': 'myExportImageTask',
 'creation_timestamp_ms': 1662080083647,
 'update_timestamp_ms': 1662080098113,
 'start_timestamp_ms': 1662080091975,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/17rvGzJEo1cwuvqCa6RQAXi5uMBEiVw6G'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 0.6657026410102844,
 'id': '2SUNJCKW7I7TPFEUE5MBOBBN',
 'name': 'projects/earthengine-legacy/operations/2SUNJCKW7I7TPFEUE5MBOBBN'}